<a href="https://colab.research.google.com/github/ratmcu/wiki_ner/blob/master/wiki_conll_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install wget
import os
import wget
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
try:
    import colabimport
except:
    colabimporturl = 'https://github.com/ratmcu/colaboratory_import/raw/master/colabimport.py'
    filename = colabimporturl.split("/")[-1].split("?")[0]
    if os.path.isfile(filename):
        os.remove(filename)
    wget.download(colabimporturl)
    import colabimport
    
colabimport.get_notebook('https://github.com/ratmcu/wiki_ner/blob/master/conll_tagged_ne.ipynb?raw=true')
from conll_tagged_ne import WikiConLLTagger
class HashableTupleAnnotations(tuple):
    def __hash__(self):
        return hash(tuple(sorted([self[0:1],self[1:2]])))
class hashabledict(dict):
    def __hash__(self):
        return hash(tuple(sorted(self.items())))
!pip install spacy
!python -m spacy download en_core_web_sm
import spacy
nlp = spacy.load("en_core_web_sm")
import re
!pip install pyahocorasick
!pip install fuzzyset
from ahocorasick import Automaton
import fuzzyset
from operator import itemgetter, attrgetter
import pandas as pd
import pickle
import urllib
from bs4 import BeautifulSoup
import numpy as np
import json

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=690365ad63d6e764d0c4bf252c8c919c0ea5a7178f9664e0c60f48cd5a27e1ad
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
importing Jupyter notebook from conll_tagged_ne.ipynb
importing Jupyter notebook from reusable_annotator.ipynb
     |████████████████████████████████| 317kB 2.7MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81710 sha256=9c16fd1098f34c7e91d841e9cd1d83b0ccf8aca87fd16d8e786a16db90b0835b
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick
     |████████████████████████████████| 378kB 2.7MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
  Created wheel for fuzzyset: filename=fuzzyset-0.0.19-cp36-cp36m-linux_x86_64.whl size=167541 sha256=aae1e4d2b844ddccf289cd01d390f30d6

In [0]:
#experiment_code
# tagger = WikiConLLTagger('https://en.wikipedia.org/wiki/Barack_Obama')
tagger = WikiConLLTagger('https://en.wikipedia.org/wiki/Donald_Trump')
# print(tagger.get_metadata())
# with open('meta_data.pkl', 'wb') as f:
#     pickle.dump(tagger.get_metadata(), f)

DEBUG:root:{'Vice President': ['Mike Pence'], 'Preceded by': ['Barack Obama'], 'Born': ['Donald John Trump', '', '1946-06-14', ')', 'June 14, 1946', '', 'Queens', ',', 'New York City'], 'Political party': ['Republican', '2009–2011, 2012–present)'], 'Other politicalaffiliations': ['', 'Democratic', '1987, 2001–2009)', '', 'Reform', '', '', 'Independent', '', ''], 'Spouse(s)': ['', 'Ivana Zelníčková', '', 'm.', '', '1977', ';', 'div.', '', '1992', ')', '', 'Marla Maples', '', 'm.', '', '1993', ';', 'div.', '', '1999', ')', '', 'Melania Knauss', '', 'm.', '', '2005', ')', ''], 'Children': ['', 'Donald Jr.', '', 'Ivanka', '', 'Eric', '', 'Tiffany', '', 'Barron', ''], 'Parents': ['Fred Trump', 'Mary Anne MacLeod'], 'Relatives': ['Trump family'], 'Residence': ['', 'White House', '', '', 'Trump Tower', '', '', 'Full list', ''], 'Education': ['The Wharton School', '', 'BS', 'in', 'Econ.', ')'], 'Occupation': ['Politician', 'businessman', 'real-estate developer', 'television personality'], 'Net

info card is scraped successfully


In [0]:
#experiment_code
df = pd.DataFrame(data = tagger.place_tags())
df.to_csv(r'conll_annot.csv', index = None, header=True)

DEBUG:root:Donald John Trump
--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.6/logging/__init__.py", line 994, in emit
    msg = self.format(record)
  File "/usr/lib/python3.6/logging/__init__.py", line 840, in format
    return fmt.format(record)
  File "/usr/lib/python3.6/logging/__init__.py", line 577, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.6/logging/__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()


In [0]:
wget.download('https://github.com/ratmcu/wiki_ner/blob/master/president_list.pkl?raw=true')
category = 'politicians'

## pipeline this to accomodate other categories

with open('president_list.pkl', 'rb') as f:
    prsdnt_list = pickle.load(f)
print(prsdnt_list)
for cntry in prsdnt_list:
    for prsdnt in cntry['presidents']:
        if cntry['country'] and prsdnt['name']:
            path = 'dataset' + '/' + category + '/' + cntry['country'] + '/' +  prsdnt['name']
        else:
            continue
        if not os.path.exists(path):
            os.makedirs(path)
        else:
            continue
        try:
            tagger = WikiConLLTagger(prsdnt['url'])
            df = pd.DataFrame(data = tagger.place_tags())
            df.to_csv(r'{0}/conll_tagged.csv'.format(path), index = None, header=True)
        except:
            print("failed to annotate the page!")
            print(prsdnt['url'])
            
import tarfile
tar =  tarfile.open('dataset.tar.gz', mode='w:gz')
tar.add('dataset')
tar.close()    


In [0]:
import tarfile
tar =  tarfile.open('metadata.tar.gz', mode='w:gz')
tar.add('metadata')
tar.close()

In [0]:
quote_page = 'https://en.wikipedia.org/w/index.php?title=Category:Possibly_living_people'
# quote_page = 'https://en.wikipedia.org/wiki/Category:Living_people'
# page = urllib.request.urlopen(quote_page)
# soup = BeautifulSoup(page, 'html.parser')

def stack_category_links(url, links):
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    content_big = soup.find('div', attrs={'class': 'mw-content-ltr'}) # or infobox biography vcard
    link_list = content_big.find('div', attrs={'class': 'mw-category'}) # or infobox biography vcard
    links.extend(link_list.find_all('a'))
    next_page = content_big.next_sibling.next_sibling.a.get('href')
    try:
        stack_category_links(next_page, links)
    except:
        return links
    return links

links = []
stack_links(quote_page, links)

In [0]:
quote_page = 'https://en.wikipedia.org/w/index.php?title=Category:Possibly_living_people'
page = urllib.request.urlopen(quote_page)
soup = BeautifulSoup(page, 'html.parser')
content_big = soup.find('div', attrs={'class': 'mw-content-ltr'}) # or infobox biography vcard
links = []
link_list = content_big.find('div', attrs={'class': 'mw-category'}) # or infobox biography vcard
links.extend(link_list.find_all('a'))
next_page = content_big.next_sibling.next_sibling.a.get('href')
next_page

In [0]:
quote_page = 'https://en.wikipedia.org/w/index.php?title=Category:Living_people'
page = urllib.request.urlopen(quote_page)
soup = BeautifulSoup(page, 'html.parser')
links = []

while True:
#     try:
    content_big = soup.find('div', attrs={'class': 'mw-content-ltr'}) # or infobox biography vcard
    link_list = content_big.find('div', attrs={'class': 'mw-category'}) # or infobox biography vcard
    links.extend(link_list.find_all('a'))
    print(len(links))
    next_page = None
    for aa in soup.find_all('a'):
        if aa.text == 'next page':
            next_page = aa.get('href')
            print(aa.get('href'))
            page = urllib.request.urlopen('https://en.wikipedia.org' + next_page)
            soup = BeautifulSoup(page, 'html.parser')
            break
    if next_page == None:
        break
#     except:
#         break
# links

# Mass Dataset 

choosing pages

In [0]:
import multiprocessing
import os
import time
# split a list into evenly sized chunks
!pip install wget
import wget
import logging
import pickle
import ast
import pandas as pd
import numpy as np
import urllib
from bs4 import BeautifulSoup
import traceback
import re

try:
    import colabimport
except:
    colabimporturl = 'https://github.com/ratmcu/colaboratory_import/raw/master/colabimport.py'
    filename = colabimporturl.split("/")[-1].split("?")[0]
    if os.path.isfile(filename):
        os.remove(filename)
    wget.download(colabimporturl)
    import colabimport
colabimport.get_notebook('https://github.com/ratmcu/wiki_ner/blob/master/reusable_annotator.ipynb?raw=true')
colabimport.get_notebook('https://github.com/ratmcu/wiki_ner/blob/master/info_box.ipynb?raw=true')
# import io, os, sys, types
from reusable_annotator import PageContents
from info_box import InfoCard, PrivateEntities
# exit()
if not os.path.isfile('./Living_people.pkl'):
    wget.download('https://github.com/ratmcu/wiki_ner/blob/master/Living_people.pkl?raw=true')

logdir = 'data_frames'
if not os.path.exists(logdir): os.makedirs(logdir)
random_test_params = { 'test':True, 'samples': 40}

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=5a329a55f96457a263b6ecef1ad978960517d0a85909b830690a98d96d70e04d
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
importing Jupyter notebook from reusable_annotator.ipynb
     |████████████████████████████████| 317kB 2.7MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81712 sha256=4d78feb6d8556be22ada684a1f1d3632f1aa557be27c8c2b83e05999431be2a9
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick
     |████████████████████████████████| 378kB 2.8MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
  Created wheel for fuzzyset: filename=fuzzyset-0.0.19-cp36-cp36m-linux_x86_64.whl size=167540 sha256=0a1412cfa0e93fa70b0107cfebe952579439ae444f5f41c8022570ba3ae90ea0
  Stored in directory:

In [0]:
def add_wiki_prefix(func):
    def wrapper(url):
        url = 'http://en.wikipedia.org' + url
        return func(url)
    return wrapper

@add_wiki_prefix
def getEntityPresence(url):
    try:
        urllib.request.urlopen(url, timeout = 5)
    except:
        raise Exception('page not found!!!!!!!!!')
    pg = PageContents(url)
    
    if not pg.table:
        raise Exception('unable to find a info table in the page')
# pg = PageContents('https://en.wikipedia.org/wiki/Donald_Trump')
    info_table = InfoCard(pg).info_table
    # entities = PrivateEntities(info_card).entity_dict
    # key_dict = {'NAME', 'BIRTH_DATE', 'CHILDREN', 'SPOUSES', 'PARENTS', 'EDUCATION'}
    # ret = sum([True for key in entities.keys() 
    #            if len(entities[key])>1 and len(entities[key]) != 0
    #                                    and key in key_dict]
    ners =   [key   for key in info_table.keys() 
              if len(info_table[key])>1 and len(info_table[key]) != 0]     

    # print(entities)
    return ners

def getHealthyPages(url_list):
    page_df =  pd.DataFrame()    
    for i, link in enumerate(url_list):
        print(i, link)
        try:
            ners = getEntityPresence(link) 
            page_df = page_df.append(pd.DataFrame(np.ones((1,len(ners)), dtype=np.int64), columns=ners).join(pd.DataFrame({'link':[link]})))
            # page_df = page_df.join(pd.DataFrame({'link':[link]}))
        # except:
        #     print('failed to scrape the page: {0}'.format(link))
        except Exception as e:
            print(traceback.format_exc())
    return page_df
    
@add_wiki_prefix
def get_category_list(url):
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    catlink_box = soup.find('div', attrs={'class': 'mw-normal-catlinks'}) # or infobox biography vcard
    return [ li.text for li in catlink_box.find_all('li')]

def create_category_table(url_list):
    category_df =  pd.DataFrame()
    for i, link in enumerate(url_list):
        print(i, link)
    #     cats = [category for category in get_category_list(str(link))]
        cats = get_category_list(str(link))
        df_cats =  pd.DataFrame(np.ones((1,len(cats)), dtype=np.int64), columns=cats)
        category_df = category_df.append(df_cats, sort=True)
    return category_df

with open('Living_people.pkl', 'rb') as f:
    js = pickle.load(f)
# js = json.load(js)    
links_js = ast.literal_eval(js)

def chunks(l, n):
    return [l[i:i+n] for i in range(0, len(l), n)]
    
def do_job(in_queue, queue):
    while True:
            #try:
            #    index, url_list = saver_queue.get(block=False)
            #    queue.put((index, create_category_table(url_list)))   
            #except:
            #    pass
            #if not queue.empty():
            index, url_list = in_queue.get()
            # getHealthyPages(url_list)
            queue.put((index, getHealthyPages(url_list)))   
            # print('loaded a df {0}'.format(index))
            if in_queue.empty(): return
        
def save_chunk_to_disk(saver_queue, event):
    while True:
            try:
                index, df = saver_queue.get(block=True, timeout=1) 
                df.to_csv(logdir + '/' + 'ners_found_frame_{0}.csv'.format(index))    
                print('saved file: ners_found_frame_{0}.csv'.format(index))
            except:
                pass
            if event.is_set(): break
    print('done saving data frames to the disk')

def dispatch_jobs(data, worker_number):
    category_data_frame =  pd.DataFrame()
    total = len(data)
    chunk_size = min(100,int(len(data)/worker_number)) #total // job_number
    #chunk_size = total // worker_number
    slice = chunks(data, chunk_size)

    saver_queue = multiprocessing.Queue(1000)
    event = multiprocessing.Event()
    disk_save_mp = multiprocessing.Process(target=save_chunk_to_disk, args=(saver_queue, event))
    disk_save_mp.start()  
    workers = []
    in_queues = []
    
    print('queued data')    
    in_queue = multiprocessing.Queue(10000)
    for worker in range(worker_number):
        #in_queue = multiprocessing.Queue(1000)
        #in_queues.append(in_queue)
        j = multiprocessing.Process(target=do_job, args=(in_queue, saver_queue))
        workers.append(j)
    
    queue_number = 0
    
    # adding the data to the input queues
    for i, s in enumerate(slice):
#         print(i)
        in_queue.put((i, s))
        #in_queues[queue_number].put((i, s))
        #queue_number += queue_number
        #if queue_number == len(in_queues):
        #    queue_number = 0
        
    print('workers created')    
    
    for worker in workers:
        # time.sleep(1)
        worker.start()
    # disk_save_mp.start()    
    print('workers started')
    queue_empty = False
    running = any(p.is_alive() for p in workers)
    while running:
        #save_chunk_to_disk(saver_queue.get(block=True))
        running = any(p.is_alive() for p in workers)
    print('all items in in_queue are done processed by workers')
    while not saver_queue.empty(): pass    
    event.set()
    disk_save_mp.join()  
    return

if random_test_params['test'] == True:
    import random
    data = random.choices(links_js, k=random_test_params['samples'])
else:
    data = link_js

start_time = time.perf_counter()
print(os.cpu_count())
cpu_count = os.cpu_count()
category_data_frame = dispatch_jobs(data, cpu_count)
end_time = time.perf_counter()      # 2
run_time = end_time - start_time 
print(run_time)


# Scoring pages and choosing the best ones


## selecting the row names to be scored

In [0]:
#filter the row entry
def rowEntryFilter(lst):
    entries = set()
#     reg = re.compile('[0-9]+$', re.MULTILINE)
#     reg = re.compile('')
#     reg = re.compile('[0-9*]')
    for entry in lst:
#         match = reg.match(entry)
        if any(d in entry for d in'0123456789'): # remove the clutter a bit by removing rows with digits
            print(entry)
            continue  
        entries.add(entry)
    return entries

In [0]:
import tarfile
if not os.path.exists('900k_living_people_ner_presence.tar.gz'):
    wget.download('https://github.com/ratmcu/wiki_ner/blob/master/900k_living_people_ner_presence.tar.gz?raw=true')
tar = tarfile.open('900k_living_people_ner_presence.tar.gz', mode='r')
tar.extractall('./900k_living_people_ner_presence')
tar.close()

In [0]:
paths = sorted([os.path.join(f[0], name) for f in os.walk('./900k_living_people_ner_presence') 
                if len(f[2])!=0 for name in f[2] if os.path.splitext(name)[-1] == '.csv'],
               key=lambda path: int(path.split('_')[-1].split('.')[0]))

In [0]:
paths = sorted([os.path.join(f[0], name) for f in os.walk('./900k_living_people_ner_presence') 
                if len(f[2])!=0 for name in f[2] if os.path.splitext(name)[-1] == '.csv'],
               key=lambda path: int(path.split('_')[-1].split('.')[0]))
paths = paths[:8201]
# int(paths[0].split('_')[-1].split('.')[0])
# presence_frames_raw =  pd.DataFrame()
entity_rows = set()
for path in paths:
#     df = pd.read_csv(path).fillna(0)
    df = pd.read_csv(path)
    entity_rows.update(df.columns.tolist())
#     presence_frames_raw = presence_frames_raw.append(df,ignore_index = True)
# presence_frames_raw = presence_frames_raw.fillna(0)
# 'Home\xa0town' in set(presence_frames_raw.columns.tolist())

In [0]:
ss = rowEntryFilter(entity_rows)
with open('entity_rows.pkl', 'wb') as f:
    pickle.dump(ss, f)

In [0]:
if not os.path.exists('entity_rows.pkl'):
    wget.download('https://github.com/ratmcu/wiki_ner/blob/master/entity_rows.pkl?raw=true')
with open('entity_rows.pkl', 'rb') as f:
    ss = pickle.load(f)
fltrd = rowEntryFilter(ss)
with open('entity_rows.txt', 'w') as f:
    for entry in fltrd:
        f.writelines(entry+'\n')

In [0]:
#if the rows mentioned in the following text following exists in a link score of one is given to each presence
if not os.path.exists('selected_entity_rows.txt'):
    wget.download('https://github.com/ratmcu/wiki_ner/blob/master/selected_entity_rows.txt?raw=true')
selected_entity_rows = open('selected_entity_rows.txt', 'r').read().splitlines()
selected_entity_rows = set(selected_entity_rows)
selected_entity_rows.add('Alma\xa0mater')
selected_entity_rows
# chosen_entity_rows = {'Alma mater', 'Alma\xa0mater', 'Born', 'Born:', 
#                 'College', 'Education','Parent(s)', 'Parents',
#                 'High school', 'High school:', 'Spouse(s)',
#                 'Place of birth', }


In [0]:
def scorePages(page_summary):
    '''Give a score to each entry by the columns with the ones'''
    score_list = []
#     selected_entity_rows = {'Alma mater', 'Alma\xa0mater', 'Born', 'Born:', 
#                 'College', 'Education','Parent(s)', 'Parents',
#                 'High school', 'High school:', 'Spouse(s)',
#                 'Place of birth'}
    
    for idx, row in enumerate(page_summary.iterrows()):
        frame = row[1]
        score = 0
        for key in frame.keys():
            if key in {'link'}:
                continue
#             print(frame[key],key)
            elif key in selected_entity_rows:
                score = score + frame[key]
# #         score_df = score_df.append(pd.DataFrame({'index': [idx], 'score':score, 'NAME': name }))
# #         score_df = score_df.append(pd.DataFrame({'INDEX': [idx], 'SCORE':score }))    
        score_list.append(score)
    return pd.DataFrame({'SCORE': score_list })

# paths = paths[:101]
paths = paths[:8201]
presence_score_df =  pd.DataFrame()
for path in paths:
    df = pd.read_csv(path).fillna(0).iloc[:,1:]
#     print(df['link'])
    df = pd.DataFrame({'LINK': df['link'] }).assign(SCORE=scorePages(df))
    presence_score_df = presence_score_df.append(df, ignore_index = True)
presence_score_df = presence_score_df.sort_values(by='SCORE', ascending = False)
# presence_frames.to_csv('presence_frames.csv')
# presence_frames.drop_duplicates(subset='link')
# from inspect import currentframe, getsourcelines, getframeinfo
# frameinfo  = getframeinfo(currentframe())
# framelines = getsourcelines(currentframe())
# ll = [print(line[:-1]) for line in framelines[0]]

In [0]:
presence_score_df.to_csv('entity_presence_score.csv')

## Analysis of the Entity presence inside the info box

In [0]:
if not os.path.exists('entity_presence_score.csv'):
    wget.download('https://github.com/ratmcu/wiki_ner/blob/master/entity_presence_score.csv?raw=true')
#sanity check of the score csv
# for path in paths:
#     df = pd.read_csv(path)
#TBD whole sanity check is ambigous due to missing links in the middle, 
#so a dummy frame per every failed page has to be inserted for clarity
entity_presence_score_df = pd.read_csv('entity_presence_score.csv')
# sanity_check_samples_1 = entity_presence_score_df.sample(n=10)
# print(sanity_check_samples_1.iloc[:,0].iloc[0], sanity_check_samples_1.iloc[:,1].iloc[0])
# print(sanity_check_samples_1.iloc[0]['LINK'], sanity_check_samples_1.iloc[0][0])
# sanity_check_samples_2 = pd.read_csv(paths[sanity_check_samples_1.iloc[0][0]//100])
# print(sanity_check_samples_2.iloc[sanity_check_samples_1.iloc[0][0]%100]['link'])

In [0]:
import matplotlib.pyplot as plt
entity_presence_score_df.iloc[:,1:].plot.hist(by='SCORE',bins=7)
histogram_df = entity_presence_score_df.iloc[:,1:].groupby('SCORE')
# entity_presence_score_df.plot.hist(by='SCORE',bins = max(entity_presence_score_df['SCORE']), alpha=0.5)

In [0]:
# histogram_df.groupby('SCORE').count()
histogram_df=entity_presence_score_df.iloc[:,1:].groupby('SCORE').count()
histogram_df

making a list of all column names to consider, 
score and choose the ones with all the entitites,

add new table entries to the entity extractor

also add the negative example marking in to the dataset,
also leave neutral unmarked and entitites marked,

look into sentence tokenization as well


## Testing entity extraction from the pages with 7 relavant rows in the info box

In [0]:
logger.setLevel(logging.INFO)
indices = entity_presence_score_df.iloc[:,1:].groupby('SCORE').groups[7.0]
for index in indices:
    url = 'http://en.wikipedia.org'+entity_presence_score_df.iloc[index]['LINK']
    try:
        urllib.request.urlopen(url, timeout = 5)
    except:
        raise Exception('page not found!!!!!!!!!')
    pg = PageContents(url)
    
    if not pg.table:
        raise Exception('unable to find a info table in the page')
# pg = PageContents('https://en.wikipedia.org/wiki/Donald_Trump')
    else:
        print(url)
        info_card = InfoCard(pg)
        print(info_card.info_table)
        print(PrivateEntities(info_card).entity_dict)

In [0]:
import random
# data = random.choices(links_js, k=random_test_params['samples'])
indices_list = entity_presence_score_df.iloc[:,1:].groupby('SCORE').groups[6.0]

indices_list = random.choices(indices_list, k=10)
for index in indices_list:
    url = 'http://en.wikipedia.org'+entity_presence_score_df.iloc[index]['LINK']
    try:
        urllib.request.urlopen(url, timeout = 5)
    except:
        raise Exception('page not found!!!!!!!!!')
    pg = PageContents(url)
    
    if not pg.table:
        raise Exception('unable to find a info table in the page')
# pg = PageContents('https://en.wikipedia.org/wiki/Donald_Trump')
    else:
        print(url)
        info_card = InfoCard(pg)
        print(info_card.info_table)
        print(PrivateEntities(info_card).entity_dict)

In [0]:
import os
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
colabimport.get_notebook('https://github.com/ratmcu/wiki_ner/blob/master/conll_tagged_ne.ipynb?raw=true')
from conll_tagged_ne import WikiConLLTagger

# Dataset creation

In [0]:
logger.setLevel(logging.INFO)
import random
# data = random.choices(links_js, k=random_test_params['samples'])
indices_list = entity_presence_score_df.iloc[:,1:].groupby('SCORE').groups[4.0]

indices_list = random.choices(indices_list, k=10)
start_time = time.perf_counter()
for index in indices_list:
    url = 'http://en.wikipedia.org'+entity_presence_score_df.iloc[index]['LINK']
    try:
        urllib.request.urlopen(url, timeout = 5)
    except Exception as e:
        print(traceback.format_exc())
        print("failed to annotate the page!")
        continue
        
    pg = PageContents(url)
    print(url.split('/')[-1])
    if not pg.table:
        raise Exception('unable to find a info table in the page')
    else:
        print(url)
        info_card = InfoCard(pg)
        print(info_card.info_table)
        print(PrivateEntities(info_card).entity_dict)
    
#     path = 'dataset' + '/' + url.split('/')[-1]
#     print(path)
#     if not os.path.exists(path):
#         os.makedirs(path)
#     else:
#         continue
    try:
        tagger = WikiConLLTagger(url)
        df = pd.DataFrame(data = tagger.place_tags())
#         df.to_csv(r'{0}/conll_tagged.csv'.format(path), index = None, header=True)
#         summary_dict = {'URL':url}
        summary_dict = {}
        summary = tagger.tag_factory.get_tag_summary()
        [ summary_dict.update({key:list([summary[key]])}) for key in summary.keys()]
        summary_df = pd.DataFrame(data = summary_dict)
        print(summary_df)
#         summary_df.to_csv(r'{0}/summary.csv'.format(path), index = None, header=True)
#         print(tagger.tag_factory.get_tag_summary())
    except Exception as e:
        print(traceback.format_exc())
        print("failed to annotate the page!")
        print(url)
end_time = time.perf_counter()      # 2
run_time = end_time - start_time 
print(run_time)

In [0]:
import os
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
!pip install wget
import wget
try:
    import colabimport
except:
    colabimporturl = 'https://github.com/ratmcu/colaboratory_import/raw/master/colabimport.py'
    filename = colabimporturl.split("/")[-1].split("?")[0]
    if os.path.isfile(filename):
        os.remove(filename)
    wget.download(colabimporturl)
    import colabimport
# import importlib 
# importlib.reload(conll_tagged_ne) 
# from conll_tagged_ne import WikiConLLTagger    
colabimport.get_notebook('https://github.com/ratmcu/wiki_ner/blob/master/conll_tagged_ne.ipynb?raw=true')
from conll_tagged_ne import WikiConLLTagger
import pandas as pd
import urllib
import multiprocessing
import time
# split a list into evenly sized chunks
#!pip install wget
import wget
import pickle
import ast
import numpy as np
from bs4 import BeautifulSoup
import traceback

In [0]:
csv_url = 'https://github.com/ratmcu/wiki_ner/blob/master/entity_presence_score.csv?raw=true'
csv_filename = csv_url.split("/")[-1].split("?")[0]
if os.path.isfile(csv_filename):
        os.remove(csv_filename)
wget.download(csv_url)
entity_presence_score_df = pd.read_csv(csv_filename)

# import matplotlib.pyplot as plt
# entity_presence_score_df.iloc[:,1:].plot.hist(by='SCORE',bins=7)
# histogram_df = entity_presence_score_df.iloc[:,1:].groupby('SCORE')

indices_list = []
indices_list.extend(entity_presence_score_df.iloc[:,1:].groupby('SCORE').groups[4.0])
indices_list.extend(entity_presence_score_df.iloc[:,1:].groupby('SCORE').groups[5.0])
indices_list.extend(entity_presence_score_df.iloc[:,1:].groupby('SCORE').groups[6.0])
indices_list.extend(entity_presence_score_df.iloc[:,1:].groupby('SCORE').groups[7.0])
print(indices_list)
data = []
for index in indices_list:
    data.append((index, 'http://en.wikipedia.org' +entity_presence_score_df.iloc[index]['LINK']))
# print(data)
# exit(0)
save_dir = 'scrapes'
if not os.path.exists(save_dir): os.makedirs(save_dir)

def get_annotations(url):
    try:
#     from conll_tagged_ne import WikiConLLTagger
        tagger = WikiConLLTagger(url[1])
        df = pd.DataFrame(data = tagger.place_tags()).copy()
        summary_dict = {'URL':url[1]}
        summary = tagger.tag_factory.get_tag_summary()
        [ summary_dict.update({key:list([summary[key]])}) for key in summary.keys()]
        summary_df = pd.DataFrame(data = summary_dict)
        print(summary_df)
        return url[0], df, summary_df, url[1].split('/')[-1]
#     return url[0], pd.DataFrame(), pd.DataFrame(), url[1].split('/')[-1]

    except Exception as e:
        print(traceback.format_exc())
        print("failed to annotate the page!")
        print(url)    

def chunks(l, n):
    return [l[i:i+n] for i in range(0, len(l), n)]

def do_job(in_queue, queue):
    import logging
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    while True:
        if in_queue.empty(): return
        url_list = in_queue.get()
        for url in url_list:
            queue.put(get_annotations(url))
        
def save_chunk_to_disk(saver_queue, event):
    while True:
            try:
                index, annot_df, summary_df, name = saver_queue.get(block=True, timeout=1)
                if not os.path.exists(save_dir + '/' + name): os.makedirs(save_dir + '/' + name)
                annot_df.to_csv(save_dir + '/' + name + '/' + 'annot_csv_{0}.csv'.format(index))    
                print('saved file: annot_csv_{0}.csv'.format(index))
                summary_df.to_csv(save_dir + '/' + name + '/' + 'summary_df_{0}.csv'.format(index))    
                print('saved file: summary_df_{0}.csv'.format(index))
#             except Exception as e:
#                 print(traceback.format_exc())
            except:
                pass
            if event.is_set(): break
    print('done saving files the disk')
             
def dispatch_jobs(data, worker_number):
    category_data_frame =  pd.DataFrame()
    total = len(data)
    chunk_size = 100 #total // job_number
    #chunk_size = total // worker_number
    slice = chunks(data, chunk_size)

    saver_queue = multiprocessing.Queue(1000)
    event = multiprocessing.Event()
    disk_save_mp = multiprocessing.Process(target=save_chunk_to_disk, args=(saver_queue, event))
    workers = []
    in_queues = []
    
    print('queued data')    
    in_queue = multiprocessing.Queue(10000)
    for worker in range(worker_number):
        j = multiprocessing.Process(target=do_job, args=(in_queue, saver_queue))
        workers.append(j)
    
    queue_number = 0
    
    for i, s in enumerate(slice):
        in_queue.put(s)
        
    print('workers created')    
    
    for worker in workers:
        time.sleep(1)
        worker.start()
    disk_save_mp.start()    
    print('workers started')
    queue_empty = False
    running = any(p.is_alive() for p in workers)
    while running:
        #save_chunk_to_disk(saver_queue.get(block=True))
        running = any(p.is_alive() for p in workers)
    print('all in queues drained')
    while not saver_queue.empty(): pass    
    event.set()
    disk_save_mp.join()  
    return

data = data[:10]
start_time = time.perf_counter()
print(os.cpu_count())
cpu_count = os.cpu_count()
dispatch_jobs(data, cpu_count)
end_time = time.perf_counter()      # 2
run_time = end_time - start_time 
print(run_time)
import tarfile
tar =  tarfile.open('dataset.tar.gz', mode='w:gz')
tar.add('scrapes')
tar.close()   
import tarfile
tar =  tarfile.open('dataset.tar.gz', mode='w:gz')
tar.add('scraped')
tar.close()

## working with the mass dataset

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tarfile
# ('/content/drive/My Drive/Colab Notebooks/dataset.tar.gz')
tar = tarfile.open('/content/drive/My Drive/Colab Notebooks/dataset.tar.gz', mode='r')
tar.extractall('./dataset')
tar.close()

In [0]:
import os
paths_summary = sorted([os.path.join(f[0], name) for f in os.walk('./dataset') 
                if len(f[2])!=0 for name in f[2] if os.path.splitext(name)[-1] == '.csv' and name.split('_')[0]=='summary'],
               key=lambda path: int(path.split('_')[-1].split('.')[0]))

paths_annot = sorted([os.path.join(f[0], name) for f in os.walk('./dataset') 
                if len(f[2])!=0 for name in f[2] if os.path.splitext(name)[-1] == '.csv' and name.split('_')[0]=='annot'],
               key=lambda path: int(path.split('_')[-1].split('.')[0]))

In [0]:
len(paths_annot), len(paths_summary)

(23107, 23107)

In [0]:
import pandas as pd
summary_df = pd.DataFrame()
import random
paths = random.choices(paths_summary, k=10)
# for path in paths:
for path in paths_summary[:10]:
    df = pd.read_csv(path).iloc[:,1:]
#     if df.iloc[:,1:]['BP'][0]!=0:
#         summary_df = summary_df.append(df, ignore_index = True)
    summary_df = summary_df.append(df, ignore_index = True)
print(summary_df['URL'])
summary_df.iloc[:,1:]

0    http://en.wikipedia.org/wiki/Carl_Roberts_(dip...
1    http://en.wikipedia.org/wiki/Unmesh_Bhaiyyasah...
2         http://en.wikipedia.org/wiki/Kantilal_Bhuria
3         http://en.wikipedia.org/wiki/Stephen_Colbert
4             http://en.wikipedia.org/wiki/Rubab_Sayda
5             http://en.wikipedia.org/wiki/Kees_Meeuws
6     http://en.wikipedia.org/wiki/Basori_Singh_Masram
7    http://en.wikipedia.org/wiki/Craig_Roberts_Sta...
8             http://en.wikipedia.org/wiki/Tamim_Iqbal
9             http://en.wikipedia.org/wiki/Mark_Harmon
Name: URL, dtype: object


,BD,NBD,CH,PR,SP,NPE,BP,NGPE,ED,NORG,SENTS
0,1,1,0,0,0,0,0,7,0,2,1
1,0,1,0,0,0,5,0,0,0,7,8
2,0,8,0,0,0,4,0,2,0,7,5
3,1,193,0,1,2,350,0,103,6,275,374
4,1,2,1,0,0,6,0,0,0,5,5
5,1,13,0,0,0,5,0,0,0,6,12
6,0,3,0,0,2,3,0,1,0,3,6
7,1,8,0,2,1,20,0,13,2,22,22
8,0,71,0,1,1,32,0,68,0,77,128
9,1,69,0,5,2,109,0,28,4,67,90


In [0]:
import pandas as pd
summary_df = pd.DataFrame()
import random
paths = random.choices(paths_summary, k=10)
# for path in paths:
for path in paths_summary:
    df = pd.read_csv(path).iloc[:,1:]
#     if df.iloc[:,1:]['BP'][0]!=0:
    summary_df = summary_df.append(df, ignore_index = True)
#     summary_df = summary_df.append(df, ignore_index = True)
# print(summary_df['URL'])
summary_df.iloc[:,1:].sum()

BD         19712
NBD       569829
CH         14741
PR          8531
SP         33725
NPE       723174
BP             0
NGPE      352468
ED         30978
NORG      642738
SENTS    1008915
dtype: int64

## splitting the dataset to the test/valid/train

### method 1

#### score based on the final annotations and choose the bunch with many as the training and validation set


In [0]:
# summary_df.head(10)
scored_summary = summary_df.assign(SCORE = summary_df[['BD','CH', 'PR', 'SP','BP', 'ED']].apply(lambda x: x>0, axis=1, result_type='broadcast').sum(axis=1))
scored_summary.to_csv('scored_summary.csv')

In [0]:
scored_summary_sorted = scored_summary.sort_values(by='SCORE', ascending = False)
scored_summary_sorted.groupby('SCORE').count()

,URL,BD,NBD,CH,PR,SP,NPE,BP,NGPE,ED,NORG,SENTS
SCORE,,,,,,,,,,,,
0,1574,1574,1574,1574,1574,1574,1574,1574,1574,1574,1574,1574
1,4344,4344,4344,4344,4344,4344,4344,4344,4344,4344,4344,4344
2,6284,6284,6284,6284,6284,6284,6284,6284,6284,6284,6284,6284
3,7853,7853,7853,7853,7853,7853,7853,7853,7853,7853,7853,7853
4,2745,2745,2745,2745,2745,2745,2745,2745,2745,2745,2745,2745
5,307,307,307,307,307,307,307,307,307,307,307,307


#### ...
score of 5 as test\
score of 4 as eval

In [0]:
scored_summary_sorted.groupby('SCORE').groups[5.0]

Int64Index([ 1704,  2101,  1309,   491,  1311,   486,   484,   477,  2115,
             1326,
            ...
             5416, 12459,  3253,   221,  5316, 14670,  1909,  3258,  5411,
             6280],
           dtype='int64', length=307)

In [0]:
!pip install wget
import os
import wget
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
try:
    import colabimport
except:
    colabimporturl = 'https://github.com/ratmcu/colaboratory_import/raw/master/colabimport.py'
    filename = colabimporturl.split("/")[-1].split("?")[0]
    if os.path.isfile(filename):
        os.remove(filename)
    wget.download(colabimporturl)
    import colabimport
    
colabimport.get_notebook('https://github.com/ratmcu/wiki_ner/blob/master/wiki_ner_loader.ipynb?raw=true')
from wiki_ner_loader import toConllTxt

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=99cec06905b81f33139e90881150cbb5692921f961c06577c08ad3f30d914fdc
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
importing Jupyter notebook from wiki_ner_loader.ipynb
     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 655kB 59.7MB/s 
  Created wheel for regex: filename=regex-2019.8.19-cp36-cp36m-linux_x86_64.whl size=609240 sha256=52f7ff062eab1c644dc07380abe2fff8f65e236619561381ab6dc7ebb2775058
  Stored in directory: /root/.cache/pip/wheels/90/04/07/b5010fb816721eb3d6dd64ed5cc8111ca23f97fdab8619b5be
Successfully built regex


100%|██████████| 213450/213450 [00:00<00:00, 386581.73B/s]


#### test set creation

In [0]:
#experiment_code
test_indices = scored_summary_sorted.groupby('SCORE').groups[5.0]
print(sorted(test_indices))
paths_annot = sorted([os.path.join(f[0], name) for f in os.walk('./dataset') 
                if len(f[2])!=0 for name in f[2] if os.path.splitext(name)[-1] == '.csv' and name.split('_')[0]=='annot'],
               key=lambda path: int(path.split('_')[-1].split('.')[0]))
test_paths = [path for path in paths_annot if int(path.split('_')[-1].split('.')[0]) in test_indices]
print(test_paths)
test_data_folder = './dataset_txt/test'
if not os.path.exists(test_data_folder): os.makedirs(test_data_folder)
import pandas as pd
for i, path in enumerate(test_paths):
    print(path.split('/')[-2])
    save_file = os.path.join(test_data_folder, '%s.txt' % path.split('/')[-1].split('.')[0])
    txt_path = toConllTxt(path, save_file)
    print(txt_path)
    # data_page = NerDataset(txt_path)
    # dataset.append(data_page)
    # print(len(data_page), ' ', i)
# print(len(dataset))

[34, 39, 52, 54, 58, 127, 129, 139, 163, 171, 182, 185, 186, 197, 221, 226, 276, 299, 313, 314, 325, 334, 335, 337, 342, 345, 357, 379, 388, 407, 442, 451, 453, 457, 464, 477, 484, 486, 491, 533, 535, 540, 563, 569, 575, 612, 638, 640, 694, 746, 754, 755, 759, 777, 784, 864, 873, 899, 950, 959, 974, 988, 995, 1005, 1010, 1017, 1028, 1031, 1049, 1060, 1067, 1078, 1084, 1090, 1114, 1116, 1120, 1121, 1146, 1179, 1197, 1201, 1202, 1208, 1218, 1277, 1284, 1303, 1309, 1311, 1326, 1331, 1360, 1362, 1384, 1394, 1405, 1419, 1464, 1470, 1487, 1503, 1562, 1564, 1566, 1576, 1602, 1659, 1681, 1682, 1693, 1704, 1739, 1766, 1774, 1781, 1785, 1802, 1856, 1860, 1866, 1871, 1877, 1909, 1914, 1958, 1964, 1966, 1968, 1984, 1995, 2019, 2041, 2077, 2090, 2101, 2115, 2139, 2156, 2166, 2170, 2205, 2239, 2250, 2269, 2285, 2293, 2306, 2312, 2329, 2356, 2389, 2392, 2394, 2397, 2403, 2405, 2407, 2432, 2450, 2479, 2485, 2491, 2517, 2521, 2546, 2552, 2556, 2559, 2566, 2591, 2601, 2612, 2632, 2635, 2650, 2661, 2676,

#### eval set creation

In [0]:
#experiment_code
eval_indices = scored_summary_sorted.groupby('SCORE').groups[4.0]
print(sorted(eval_indices))
paths_annot = sorted([os.path.join(f[0], name) for f in os.walk('./dataset') 
                if len(f[2])!=0 for name in f[2] if os.path.splitext(name)[-1] == '.csv' and name.split('_')[0]=='annot'],
               key=lambda path: int(path.split('_')[-1].split('.')[0]))
eval_paths = [path for path in paths_annot if int(path.split('_')[-1].split('.')[0]) in eval_indices]
print(eval_paths)
eval_data_folder = './dataset_txt/eval'
if not os.path.exists(eval_data_folder): os.makedirs(eval_data_folder)
import pandas as pd
for i, path in enumerate(eval_paths):
    print(path.split('/')[-2])
    save_file = os.path.join(eval_data_folder, '%s.txt' % path.split('/')[-1].split('.')[0])
    txt_path = toConllTxt(path, save_file)
    print(txt_path)

[3, 7, 9, 12, 14, 16, 17, 23, 24, 30, 31, 35, 38, 40, 42, 44, 45, 49, 55, 59, 64, 65, 68, 71, 74, 76, 80, 81, 85, 86, 87, 88, 92, 94, 95, 96, 99, 101, 103, 104, 106, 107, 108, 109, 112, 116, 117, 120, 121, 123, 124, 128, 131, 136, 138, 142, 143, 144, 151, 153, 155, 156, 159, 164, 166, 167, 169, 175, 181, 184, 188, 189, 199, 200, 209, 216, 220, 225, 228, 230, 232, 233, 234, 236, 243, 244, 247, 250, 251, 253, 259, 261, 262, 263, 265, 273, 281, 283, 287, 293, 295, 298, 300, 302, 303, 307, 308, 310, 312, 316, 320, 321, 322, 323, 324, 326, 327, 331, 333, 338, 341, 343, 344, 354, 356, 358, 359, 360, 362, 366, 367, 368, 369, 370, 376, 380, 381, 383, 385, 389, 392, 393, 396, 398, 399, 404, 411, 413, 416, 422, 424, 425, 427, 428, 432, 433, 437, 438, 440, 443, 449, 452, 456, 459, 467, 470, 471, 474, 478, 480, 487, 500, 501, 503, 504, 505, 513, 515, 516, 518, 522, 523, 532, 542, 546, 547, 548, 551, 556, 557, 558, 559, 570, 573, 580, 581, 582, 585, 587, 592, 602, 606, 607, 619, 633, 634, 635, 643,

In [0]:
train_indices = scored_summary_sorted.groupby('SCORE').groups[3.0]
print([key for key in scored_summary_sorted.groupby('SCORE').groups.keys()])
keys = [key for key in scored_summary_sorted.groupby('SCORE').groups.keys()]
training_indices = []
for key in keys[:4]:
    print(key)
    training_indices.extend(scored_summary_sorted.groupby('SCORE').groups[keys[key]])
len(training_indices)
training_paths = [path for path in paths_annot if int(path.split('_')[-1].split('.')[0]) in training_indices]
training_data_folder = './dataset_txt/training'
if not os.path.exists(training_data_folder): os.makedirs(training_data_folder)
import pandas as pd
for i, path in enumerate(training_paths):
    print(path.split('/')[-2])
    save_file = os.path.join(training_data_folder, '%s.txt' % path.split('/')[-1].split('.')[0])
    txt_path = toConllTxt(path, save_file)
    print(txt_path)

[0, 1, 2, 3, 4, 5]
0
1
2
3
Carl_Roberts_(diplomat)
./dataset_txt/training/annot_csv_0.txt
Unmesh_Bhaiyyasaheb_Patil
./dataset_txt/training/annot_csv_1.txt
Kantilal_Bhuria
./dataset_txt/training/annot_csv_2.txt
Rubab_Sayda
./dataset_txt/training/annot_csv_4.txt
Kees_Meeuws
./dataset_txt/training/annot_csv_5.txt
Basori_Singh_Masram
./dataset_txt/training/annot_csv_6.txt
Tamim_Iqbal
./dataset_txt/training/annot_csv_8.txt
Gajendra_Singh_Rajukhedi
./dataset_txt/training/annot_csv_10.txt
Gajendra_Patel
./dataset_txt/training/annot_csv_11.txt
Ajay_Arjun_Singh
./dataset_txt/training/annot_csv_13.txt
Ho_Kah_Leong
./dataset_txt/training/annot_csv_15.txt
Harry_Enfield
./dataset_txt/training/annot_csv_18.txt
Nicolas_Kazadi
./dataset_txt/training/annot_csv_19.txt
Alastair_Cutting
./dataset_txt/training/annot_csv_20.txt
William_Cavendish,_Earl_of_Burlington
./dataset_txt/training/annot_csv_21.txt
Mar%C3%ADa_Luisa_Piraquive
./dataset_txt/training/annot_csv_22.txt
Sean_Marsden
./dataset_txt/training/a

In [0]:
import tarfile
tar =  tarfile.open('dataset_txt.tar.gz', mode='w:gz')
tar.add('./dataset_txt')
tar.close()

In [0]:
!cp dataset_txt.tar.gz /content/drive/My\ Drive/Colab\ Notebooks/